<center><h3 style="color:green">Importing Required Libraries</h3></center>

In [1]:
import pandas as pd
import joblib
from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv(r"C:\Users\teja1\Downloads\MY2022 Fuel Consumption Ratings.csv")
df.head()

,Model Year,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption (City (L/100 km),Fuel Consumption(Hwy (L/100 km)),Fuel Consumption(Comb (L/100 km)),Fuel Consumption(Comb (mpg)),CO2 Emissions(g/km),CO2 Rating,Smog Rating
0,2022,Acura,ILX,Compact,2.4,4,AM8,Z,9.9,7.0,8.6,33,200,6,3
1,2022,Acura,MDX SH-AWD,SUV: Small,3.5,6,AS10,Z,12.6,9.4,11.2,25,263,4,5
2,2022,Acura,RDX SH-AWD,SUV: Small,2.0,4,AS10,Z,11.0,8.6,9.9,29,232,5,6
3,2022,Acura,RDX SH-AWD A-SPEC,SUV: Small,2.0,4,AS10,Z,11.3,9.1,10.3,27,242,5,6
4,2022,Acura,TLX SH-AWD,Compact,2.0,4,AS10,Z,11.2,8.0,9.8,29,230,5,7


In [3]:
df.drop(columns=['Model Year','Fuel Consumption (City (L/100 km)','Fuel Consumption(Hwy (L/100 km))','Fuel Consumption(Comb (mpg))','CO2 Rating'],inplace=True)
# Drop unwanted columns

### Rename columns

In [4]:
l=["make","model","vehicle_class","engine","cylinder","transmission","fuel","consumption","co2","smokerating"]
df.rename(columns={i:j for i,j in zip(df.columns,l)},inplace=True)

### Seperated categorial and numberial data

In [5]:
qualititive = []
quantitive = []

for i in df.columns:
    if type(df[i].iloc[0]) == str:
        qualititive.append(i)
    else:
        quantitive.append(i)  

In [6]:
print("The Qualitative data :\n {},\n The Quantitave data :\n {},\n".format(qualititive,quantitive))

The Qualitative data :
 ['make', 'model', 'vehicle_class', 'transmission', 'fuel'],
 The Quantitave data :
 ['engine', 'cylinder', 'consumption', 'co2', 'smokerating'],



### Preparing data

In [7]:
df=df[['make', 'model', 'vehicle_class', 'transmission', 'fuel','engine', 'cylinder', 'co2', 'smokerating', 'consumption']]

In [8]:
x = df.drop("consumption",axis=1)
y = df["consumption"]

In [9]:
x

,make,model,vehicle_class,transmission,fuel,engine,cylinder,co2,smokerating
0,Acura,ILX,Compact,AM8,Z,2.4,4,200,3
1,Acura,MDX SH-AWD,SUV: Small,AS10,Z,3.5,6,263,5
2,Acura,RDX SH-AWD,SUV: Small,AS10,Z,2.0,4,232,6
3,Acura,RDX SH-AWD A-SPEC,SUV: Small,AS10,Z,2.0,4,242,6
4,Acura,TLX SH-AWD,Compact,AS10,Z,2.0,4,230,7
...,...,...,...,...,...,...,...,...,...
941,Volvo,XC40 T5 AWD,SUV: Small,AS8,Z,2.0,4,219,5
942,Volvo,XC60 B5 AWD,SUV: Small,AS8,Z,2.0,4,219,5
943,Volvo,XC60 B6 AWD,SUV: Small,AS8,Z,2.0,4,232,7
944,Volvo,XC90 T5 AWD,SUV: Standard,AS8,Z,2.0,4,236,5


In [10]:
y

0       8.6
1      11.2
2       9.9
3      10.3
4       9.8
       ... 
941     9.4
942     9.4
943     9.9
944    10.1
945    10.8
Name: consumption, Length: 946, dtype: float64

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.5,random_state=33)

In [13]:
x_train.shape,y_train.shape

((473, 9), (473,))

In [14]:
{i:x[i].nunique() for i in x.columns}

{'make': 39,
 'model': 715,
 'vehicle_class': 14,
 'transmission': 23,
 'fuel': 4,
 'engine': 36,
 'cylinder': 8,
 'co2': 242,
 'smokerating': 5}

In [15]:
{i:x_train[i].nunique() for i in x_train.columns}

{'make': 39,
 'model': 415,
 'vehicle_class': 14,
 'transmission': 22,
 'fuel': 4,
 'engine': 36,
 'cylinder': 7,
 'co2': 198,
 'smokerating': 5}

In [16]:
{i:x_test[i].nunique() for i in x_train.columns}

{'make': 38,
 'model': 404,
 'vehicle_class': 14,
 'transmission': 22,
 'fuel': 4,
 'engine': 33,
 'cylinder': 8,
 'co2': 196,
 'smokerating': 5}

In [17]:
for i in ["make","model"]:
    print("i :",set(x_train[i].values)-set(x_test[i].values))

i : {'FIAT'}
i : {'RDX SH-AWD', 'Cherokee 4X4 Active Drive Lock', 'QX60 AWD', '500X AWD', 'AMG E 53 4MATIC+ Sedan', 'Tacoma 4WD D-Cab TRD Off-Road/PRO', 'S 580 4MATIC LWB Sedan', 'F-150 Hybrid 4X4', 'Altima AWD', 'RC F', 'Defender 90 5.0L V8', 'Sportage AWD', 'Versa', 'S60 B5 AWD', 'Trailblazer', 'Vantage V8', 'Charger Widebody (MDS)', 'Sienna AWD', 'Tundra 4WD', '911 Carrera', 'Panamera 4S', 'Gladiator 4X4', 'F-TYPE P450 Coupe AWD', 'F-150 Raptor 37 4X4 (Without Stop-Start)', 'Impreza 4-Door AWD', 'RDX SH-AWD A-SPEC', 'Cayenne S', 'Wagoneer 4X4', 'AMG E 53 4MATIC+ Cabriolet', 'Kicks', 'Qashqai AWD', 'Cullinan Black Badge', 'AMG GLE 53 4MATIC+ SUV', 'Tucson', 'Sentra SR', 'Wrangler JL Unlimited 4X4 eTorque', 'Traverse AWD', 'Accord Hybrid', 'Panamera', 'CLS 450 4MATIC Coupe', 'S3 Sedan quattro', 'Z4 sDrive30i', 'TT Roadster 45 TFSI quattro', 'Macan S', 'Sorento AWD', 'TTS Coupe quattro', 'M8 Coupe Competition', 'M4 Competition Coupe M xDrive', 'Sienna', 'Sportage', 'Santa Fe Hybrid', '

In [18]:
len(set(x_train["model"].values)-set(x_test["model"].values))

311

In [19]:
one = OneHotEncoder(sparse=False)

In [20]:
x_train.iloc[:,:5]

,make,model,vehicle_class,transmission,fuel
857,Subaru,Impreza 5-Door AWD,Station wagon: Small,M5,X
594,Lexus,LS 500 AWD,Mid-size,AS10,Z
708,MINI,Cooper S Convertible,Minicompact,AM7,Z
264,Dodge,Challenger SRT Hellcat,Mid-size,A8,Z
452,Hyundai,Elantra,Mid-size,AM7,X
...,...,...,...,...,...
658,Mercedes-Benz,AMG C 43 4MATIC Coupe,Subcompact,A9,Z
578,Land Rover,Range Rover Sport P360,SUV: Standard,AS8,Z
728,Nissan,Kicks,Mid-size,AV,X
391,GMC,Sierra 4WD,Pickup truck: Standard,A8,X


In [21]:
x_dup =pd.DataFrame(one.fit_transform(x_train.iloc[:,:5]),columns=one.get_feature_names_out())
x_dup

C:\Users\teja1\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,make_Acura,make_Alfa Romeo,make_Aston Martin,make_Audi,make_BMW,make_Bentley,make_Bugatti,make_Buick,make_Cadillac,make_Chevrolet,...,transmission_AV6,transmission_AV7,transmission_AV8,transmission_M5,transmission_M6,transmission_M7,fuel_D,fuel_E,fuel_X,fuel_Z
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
470,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [22]:
x_train.drop(columns=["make","model","vehicle_class","transmission","fuel"],inplace=True)


In [23]:
x_train.reset_index(drop="first",inplace=True)
x_train

,engine,cylinder,co2,smokerating
0,2.0,4,211,7
1,3.4,6,270,5
2,2.0,4,182,7
3,6.2,8,339,1
4,1.6,4,179,5
...,...,...,...,...
468,3.0,6,252,5
469,3.0,6,269,7
470,1.6,4,169,7
471,2.7,4,306,6


In [24]:
x_dup.reset_index(drop="first",inplace=True)

In [25]:
x_train = pd.concat([x_train,x_dup],axis=1)
x_train

,engine,cylinder,co2,smokerating,make_Acura,make_Alfa Romeo,make_Aston Martin,make_Audi,make_BMW,make_Bentley,...,transmission_AV6,transmission_AV7,transmission_AV8,transmission_M5,transmission_M6,transmission_M7,fuel_D,fuel_E,fuel_X,fuel_Z
0,2.0,4,211,7,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,3.4,6,270,5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2.0,4,182,7,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,6.2,8,339,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.6,4,179,5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,3.0,6,252,5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
469,3.0,6,269,7,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
470,1.6,4,169,7,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
471,2.7,4,306,6,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [26]:
y_train

857     9.0
594    11.2
708     7.8
264    14.5
452     7.6
       ... 
658    10.8
578    11.4
728     7.2
391    13.1
20      8.2
Name: consumption, Length: 473, dtype: float64

In [27]:
from sklearn.ensemble import RandomForestRegressor

In [28]:
regressor = RandomForestRegressor()

In [29]:
regressor.fit(x_train,y_train)

RandomForestRegressor()

In [30]:
regressor.score(x_train,y_train)

0.9969318189628216

In [31]:
x_test

,make,model,vehicle_class,transmission,fuel,engine,cylinder,co2,smokerating
623,Maserati,Levante Modena V8,SUV: Standard,A8,Z,3.8,8,349,1
176,Chevrolet,Camaro,Subcompact,AS10,X,3.6,6,251,6
904,Toyota,Sequoia 4WD,SUV: Standard,AS6,X,5.7,8,385,5
345,Ford,Mustang,Subcompact,AS10,X,5.0,8,298,3
544,Kia,Seltos,SUV: Small,AV8,X,2.0,4,182,5
...,...,...,...,...,...,...,...,...,...
70,Bentley,Flying Spur,Mid-size,AM8,Z,4.0,8,323,3
189,Chevrolet,Colorado ZR2 4WD,Pickup truck: Small,A8,X,3.6,6,329,6
310,Ford,F-150,Pickup truck: Standard,AS10,X,2.7,6,256,6
381,GMC,Canyon,Pickup truck: Small,A8,X,3.6,6,263,6


In [32]:
joblib.dump(regressor,"regressor.pkl")

['regressor.pkl']

In [34]:
oneA = OneHotEncoder(sparse=False)

In [ ]:
x_dup =pd.DataFrame(one.fit_transform(x_train.iloc[:,:5]),columns=one.get_feature_names_out())
x_dup

In [35]:
test_temp =pd.DataFrame(oneA.fit_transform(x_test.iloc[:,:5]),columns=oneA.get_feature_names_out())

C:\Users\teja1\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [36]:
x_test.drop(columns=["make","model","vehicle_class","transmission","fuel"],inplace=True)


In [37]:
x_test.reset_index(drop="first",inplace=True)
x_test

,engine,cylinder,co2,smokerating
0,3.8,8,349,1
1,3.6,6,251,6
2,5.7,8,385,5
3,5.0,8,298,3
4,2.0,4,182,5
...,...,...,...,...
468,4.0,8,323,3
469,3.6,6,329,6
470,2.7,6,256,6
471,3.6,6,263,6


In [38]:
test_temp.reset_index(drop="first",inplace=True)

In [39]:
test_temp

,make_Acura,make_Alfa Romeo,make_Aston Martin,make_Audi,make_BMW,make_Bentley,make_Bugatti,make_Buick,make_Cadillac,make_Chevrolet,...,transmission_AV6,transmission_AV7,transmission_AV8,transmission_M5,transmission_M6,transmission_M7,fuel_D,fuel_E,fuel_X,fuel_Z
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
470,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [40]:
x_test = pd.concat([x_test,test_temp],axis=1)
x_test

,engine,cylinder,co2,smokerating,make_Acura,make_Alfa Romeo,make_Aston Martin,make_Audi,make_BMW,make_Bentley,...,transmission_AV6,transmission_AV7,transmission_AV8,transmission_M5,transmission_M6,transmission_M7,fuel_D,fuel_E,fuel_X,fuel_Z
0,3.8,8,349,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,3.6,6,251,6,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5.7,8,385,5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,5.0,8,298,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2.0,4,182,5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,4.0,8,323,3,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
469,3.6,6,329,6,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
470,2.7,6,256,6,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
471,3.6,6,263,6,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [42]:
x_train.shape

(473, 498)

In [43]:
x_test.shape

(473, 486)

In [45]:
498-486

12

In [46]:
final_model = joblib.load("regressor.pkl")
final_model

RandomForestRegressor()

In [47]:
final_model.predict(x_test)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- model_1500
- model_1500 4X4
- model_1500 4X4 EcoDiesel
- model_1500 4X4 eTorque
- model_1500 Classic 4X4
- ...
Feature names seen at fit time, yet now missing:
- make_FIAT
- model_1500 4X4 TRX
- model_1500 EcoDiesel
- model_1500 HFE eTorque
- model_300
- ...


In [1]:
pwd

'C:\\Users\\teja1\\Documents'